<h1>HTML table scraping and some very basic data manipulation.<h1>
    <h2>Labeling columns.<h2>
    <h2>Filtering row based on a value in column.<h2>
    <h2>Replacing column data with specific value.<h2>

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium # map rendering library

Scraping html table directly to dataframe.pd.read_html function searches for table elements and only for tr and th rows and td elements within each tr or th element in the table. td stands for “table data”.

In [25]:
neighborhoods = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', skiprows=1)[0]


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Labeling our three columns.

In [26]:
neighborhoods.columns = ['PostalCode','Borough','Neighborhood']


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [27]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']


![](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1592784000000&hmac=iVEHs0i7XQT-u9cCV4jledu1zS0VosNkKzfuOsnY8lM)
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [28]:
neighborhoods.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()


,PostalCode,Neighborhood
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
...,...,...
98,M9N,Weston
99,M9P,Westmount
100,M9R,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,"South Steeles, Silverstone, Humbergate, Jamest..."


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [29]:
neighborhoods.Neighborhood[neighborhoods.Neighborhood=='Not assigned'] = neighborhoods.Neighborhood.replace('Not assigned',neighborhoods.Borough)

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [33]:
print(neighborhoods.shape)

(103, 3)


* Use the Geocoder package or the csv file to create the following dataframe:
![](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1592784000000&hmac=BkmKRXnNmOnaTyX0-txuVGdVpzKhMVYBGoP5rV2C4Ig)

In [35]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.columns=['PostalCode','Latitude','Longitude']
result = pd.merge(neighborhoods, coordinates, on='PostalCode')
print(result.head)


FileNotFoundError: [Errno 2] File ../input/usa-geospatial-coordinates/Geospatial_Coordinates.csv does not exist: '../input/usa-geospatial-coordinates/Geospatial_Coordinates.csv'

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[coordinates.Latitude, coordinates.Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto